# Data Cleaning - Episodes Data

This notebook will perform some data cleaning on the episodes dataframe. Changes mostly involve parsing the raw strings into a useful form for EDA and Modelling studies.

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

Quick check that we havent lossed/duplicated data somewhere, check that the number of players is 4x the number of episodes.

In [2]:
df_episodes = pd.read_csv("../data/all_episodes.csv")
df_players = pd.read_csv("../data/all_players.csv")
# sanity check episodes x 4 = players
print("Episodes: {}".format(len(df_episodes)))
print("Players: {}".format(len(df_players)))
print("Length Check: {}".format(len(df_episodes)*4 == len(df_players)))

Episodes: 2161
Players: 8644
Length Check: True


## Dataframe Inspection

In [3]:
df_episodes.head()

,Date,Episode,Team,Chaser,No. OfPlayers InFinal Chase,PrizeFund,Target,Winner,Push-backAttempts,Push-backsTaken,ChaserAccuracy,ChaserSpeed,FinalChaseVideo,Series,isCelebrity,Series &Episode
0,29/06/2009,1.0,"Lisa, Ian, Claire, Driss",Mark Labbett,2,"£20,200",18,Chaser by 0:07,7.0,3.0,75%,14.9,-,1,False,NaN
1,30/06/2009,2.0,"Bradley, Christine, Foiz, Jill",Shaun Wallace,3,"£36,000",20,Chaser by 0:02,5.0,2.0,81%,13.7,-,1,False,NaN
2,01/07/2009,3.0,"Lynn, Paul, Liz, Ray",Mark Labbett,4,"£12,400",22,Chaser by 0:08,3.0,2.0,86%,15.0,-,1,False,NaN
3,02/07/2009,4.0,"Mike, Sally, Ciaran, Bette",Shaun Wallace,3,"£16,500",20,Team by 6,8.0,5.0,70%,13.5,-,1,False,NaN
4,03/07/2009,5.0,"Allan, Lisa, Phil, Emma",Shaun Wallace,2,"£16,000",14,Chaser by 0:24,2.0,1.0,88%,10.6,-,1,False,NaN


In [4]:
df_episodes.tail()

,Date,Episode,Team,Chaser,No. OfPlayers InFinal Chase,PrizeFund,Target,Winner,Push-backAttempts,Push-backsTaken,ChaserAccuracy,ChaserSpeed,FinalChaseVideo,Series,isCelebrity,Series &Episode
2156,23/04/2023,NaN,"Christine Ohuruogu, Max George, Maisie Smith, ...",Anne Hegerty,3,"£13,000",16,Chaser by 0:42,0.0,0.0,100%,12.3,-,0,True,13.07
2157,30/04/2023,NaN,"Ortis Deley, Maggie Aderin-Pocock, Graeme Hall...",Shaun Wallace,4,"£100,000",17,Chaser by 0:31,1.0,1.0,95%,12.8,-,0,True,12.09
2158,07/05/2023,NaN,"David James, Lucrezia Millarini, Myles Stephen...",Paul Sinha,3,"£213,000",18,Chaser by 0:28,2.0,1.0,90%,13.7,-,0,True,12.10
2159,21/05/2023,NaN,"Danny John-Jules, Yinka Bokinni, Tom Bosworth,...",Jenny Ryan,3,"£17,000",17,Chaser by 0:11,3.0,2.0,86%,12.1,-,0,True,12.12
2160,09/06/2023,NaN,"THE CHASE FOR SOCCER AID: Alex Brooker, Kyle W...",Anne Hegerty,4,"£169,000",21,Team by 4,7.0,4.0,75%,14.0,-,0,True,SA 2023 *


## Drop Columns

For these studies we don't need to track information on Chaser performance, will drop columns relating to this. We also drop the "FinalChaseVideo" and "Series &Episode" columns ofr being useless to us.

In [5]:
df_episodes = df_episodes.drop(columns=['Push-backAttempts', 'Push-backsTaken', 'ChaserAccuracy', 'ChaserSpeed', 'FinalChaseVideo', 'Series &Episode'])

## Rename Columns

Rename some columns to be more easily used in code

In [6]:
df_episodes = df_episodes.rename(columns={'No. OfPlayers InFinal Chase':'NPlayersFinal'})

In [7]:
df_episodes.columns

Index(['Date', 'Episode', 'Team', 'Chaser', 'NPlayersFinal', 'PrizeFund',
       'Target', 'Winner', 'Series', 'isCelebrity'],
      dtype='object')

## Check All Unique

Simply making sure there are no duplicate episodes. Using Date, Episode and Series as the ID

In [8]:
df_episodes[df_episodes[["Date","Episode","Series"]].duplicated(keep=False)]

,Date,Episode,Team,Chaser,NPlayersFinal,PrizeFund,Target,Winner,Series,isCelebrity


## Check Player Numbers

Check how many players are listed in the "Team" column for each episode. Expect for them all to be 4.

Should be a little careful of celebrity specials often have the theme listed first:
THE CHASE FOR SOCCER AID: Dion Dublin, Lianne Sanderson, David James, Shaun Williamson

To count names, first split on ":", then split on ","

In [9]:
df_episodes["Team"].apply(lambda x: len([name for name in x.split(":")[-1].split(",") if len(name) > 0])).value_counts()

4    2161
Name: Team, dtype: int64

## Chaser Episode Count

Check how many episodes each chaser has featured in

In [10]:
# Check for all chasers encountered
df_episodes["Chaser"].value_counts()

Mark Labbett      484
Anne Hegerty      478
Shaun Wallace     455
Paul Sinha        442
Jenny Ryan        240
Darragh Ennis      61
Anne Hegerty *      1
Name: Chaser, dtype: int64

Seems this single entry "Anne Hegerty *" occured for the 1000th episode special where 5 chasers were used, only one for the final chase. For these studies it is probably best to remove this datapoint.

In [11]:
df_episodes = df_episodes[~((df_episodes["Series"] == 10) & (df_episodes["Episode"] == 54))]

## Target Score

Check target scores make sense.

Often we see "+1" is being appended when the Chaser fails to get their first question and the team is able to push back by one. For this analysis I will only reference data from before the chasers final chase, hence they should be dropped.

In [12]:
# Check for all targets
df_episodes["Target"].value_counts()

18      235
17      229
16      229
15      207
19      183
14      167
20      146
13      144
12      110
21      101
11       95
22       53
10       47
23       42
9        26
24       19
8        16
7        13
18+1      8
25        8
26        7
12+1      7
16+1      7
15+1      7
17+1      6
14+1      5
21+1      5
6         5
19+1      4
11+1      3
5         3
24+1      3
22+1      2
13+1      2
20+1      2
9+1       2
10+1      2
28        2
23+1      1
24+2      1
20+2      1
3         1
14+2      1
27        1
13+2      1
13+3      1
Name: Target, dtype: int64

In [13]:
df_episodes["Target"] = df_episodes["Target"].apply(lambda x: int(str(x).split("+")[0]))

## PrizeFund string to Float

Want to Convert the string "£20,200" into numeric "20200.00"

In [14]:
df_episodes["PrizeFund"] = df_episodes["PrizeFund"].str.replace("£","").str.replace(",","").astype('float')

In [15]:
df_episodes["PrizeFund"].head()

0    20200.0
1    36000.0
2    12400.0
3    16500.0
4    16000.0
Name: PrizeFund, dtype: float64

## Parse Chaser Win Variables

The format of the "Winner" column is:
- Chaser by 0:07
- Team by 6

Want to extract this data:
- ChaserWin (Bool)
- ChaserWinBy (int: seconds), 0 for Chaser Loss
- ChaserLoseBy (int: questions), 0 for Chaser Win

In [16]:
df_episodes["ChaserWin"] = df_episodes["Winner"].str.contains("Chaser")

# Cases when chaser wins, how many seconds left on clock?
df_episodes["ChaserWinBy"] = df_episodes["Winner"].apply(lambda x: x.split("Chaser by ")[-1])
df_episodes.loc[df_episodes["ChaserWinBy"].str.contains("Team"),"ChaserWinBy"] = "0:00"
# turns mm:ss into int seconds
df_episodes["ChaserWinBy"] = df_episodes["ChaserWinBy"].apply(lambda x: pd.to_timedelta("00:" + x).total_seconds()).astype(int)

# Cases when chaser loses, how many questions did they have left?
df_episodes["ChaserLoseBy"] = df_episodes["Winner"].apply(lambda x: x.split("Team by ")[-1])
df_episodes.loc[df_episodes["ChaserLoseBy"].str.contains("Chaser"),"ChaserLoseBy"] = "0"


In [17]:
df_episodes[["Winner", "ChaserWin", "ChaserWinBy", "ChaserLoseBy"]].head(20)

,Winner,ChaserWin,ChaserWinBy,ChaserLoseBy
0,Chaser by 0:07,True,7,0
1,Chaser by 0:02,True,2,0
2,Chaser by 0:08,True,8,0
3,Team by 6,False,0,6
4,Chaser by 0:24,True,24,0
5,Team by 1,False,0,1
6,Chaser by 0:05,True,5,0
7,Team by 2,False,0,2
8,Chaser by 0:34,True,34,0
9,Chaser by 0:47,True,47,0


Finally drop Winner string

In [18]:
df_episodes = df_episodes.drop(columns=['Winner'])

## Clean Team String

Some of the specials have labels for what the episode is themed on, will remove it for easier processing 

THE CHASE FOR SOCCER AID: Dion Dublin, Lianne Sanderson, David James, Shaun Williamson

to

Dion Dublin, Lianne Sanderson, David James, Shaun Williamson

Performed by splitting on ":" and keeping the last element, then using lstrip to remove leading spaces.

In [19]:
df_episodes["Team"] = df_episodes["Team"].apply(lambda x: x.split(":")[-1].lstrip())

In [20]:
df_episodes["Team"].tail()

2156    Christine Ohuruogu, Max George, Maisie Smith, ...
2157    Ortis Deley, Maggie Aderin-Pocock, Graeme Hall...
2158    David James, Lucrezia Millarini, Myles Stephen...
2159    Danny John-Jules, Yinka Bokinni, Tom Bosworth,...
2160    Alex Brooker, Kyle Walker, Lynsey Hipgrave, De...
Name: Team, dtype: object

# Final Inspection and Save

In [21]:
df_episodes.head()

,Date,Episode,Team,Chaser,NPlayersFinal,PrizeFund,Target,Series,isCelebrity,ChaserWin,ChaserWinBy,ChaserLoseBy
0,29/06/2009,1.0,"Lisa, Ian, Claire, Driss",Mark Labbett,2,20200.0,18,1,False,True,7,0
1,30/06/2009,2.0,"Bradley, Christine, Foiz, Jill",Shaun Wallace,3,36000.0,20,1,False,True,2,0
2,01/07/2009,3.0,"Lynn, Paul, Liz, Ray",Mark Labbett,4,12400.0,22,1,False,True,8,0
3,02/07/2009,4.0,"Mike, Sally, Ciaran, Bette",Shaun Wallace,3,16500.0,20,1,False,False,0,6
4,03/07/2009,5.0,"Allan, Lisa, Phil, Emma",Shaun Wallace,2,16000.0,14,1,False,True,24,0


In [22]:
df_episodes.tail()

,Date,Episode,Team,Chaser,NPlayersFinal,PrizeFund,Target,Series,isCelebrity,ChaserWin,ChaserWinBy,ChaserLoseBy
2156,23/04/2023,NaN,"Christine Ohuruogu, Max George, Maisie Smith, ...",Anne Hegerty,3,13000.0,16,0,True,True,42,0
2157,30/04/2023,NaN,"Ortis Deley, Maggie Aderin-Pocock, Graeme Hall...",Shaun Wallace,4,100000.0,17,0,True,True,31,0
2158,07/05/2023,NaN,"David James, Lucrezia Millarini, Myles Stephen...",Paul Sinha,3,213000.0,18,0,True,True,28,0
2159,21/05/2023,NaN,"Danny John-Jules, Yinka Bokinni, Tom Bosworth,...",Jenny Ryan,3,17000.0,17,0,True,True,11,0
2160,09/06/2023,NaN,"Alex Brooker, Kyle Walker, Lynsey Hipgrave, De...",Anne Hegerty,4,169000.0,21,0,True,False,0,4


In [23]:
df_episodes.to_csv('../data/cleaned_episodes.csv', index=False)